In [1]:
import requests
import json
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

greenhouse = "https://boards.greenhouse.io"

jobs = []


ModuleNotFoundError: No module named 'selenium'

Get Adobe jobs

In [ ]:
adobe_pages = ''
adobe_url = 'https://careers.adobe.com/us/en/c/engineering-and-product-jobs?'+adobe_pages+'s=1'
print(adobe_url)
adobe_response = requests.get(adobe_url, headers=headers)

adobe_soup = BeautifulSoup(adobe_response.content, 'html5lib')
print(adobe_soup.prettify())

# write output to a file
with open('adobe.html', 'w') as f:
    f.write(adobe_soup.prettify())

# adobe_jobs_count = adobe_soup.select('span.result-count')[0].string
# adobe_jobs_count = adobe_soup.find('span', class_='result-count').string
# print(adobe_jobs_count)
# adobe_pages_count = int(adobe_jobs_count/10)

adobe_jobs = adobe_soup.find_all('li', class_='jobs-list-item')

for i in range(1, adobe_pages_count):
    adobe_pages = 'from='+i*10+'&'
    adobe_response = requests.get(adobe_url)
    adobe_soup = BeautifulSoup(adobe_response.text, 'html.parser')
    adobe_jobs += adobe_soup.find_all('li', class_='jobs-list-item')

# get the job title from the adobe_jobs list stored in anchor tag in attribute aria-label
adobe_jobs_title = [job.find('a')['aria-label'] for job in adobe_jobs] 

# get job location from attribute data-ph-at-job-location-text
adobe_jobs_location = [job.find('a')['data-ph-at-job-location-text'] for job in adobe_jobs]

# get job link from attribute href
adobe_jobs_link = [job.find('a')['href'] for job in adobe_jobs]

# get posted date from span.job-postdate 
adobe_jobs_posted = [job.find('span', class_='job-postdate').text for job in adobe_jobs]

# get job description from p.job-description innertext
adobe_jobs_description = [job.find('p', class_='job-description').text for job in adobe_jobs]

# get job id from span.jobId innertext
adobe_jobs_id = [job.find('span', class_='jobId').text for job in adobe_jobs]

# print(adobe_jobs_title)

# write results into a json file named as jobs.json in objects list with each object containing job title, location, link, posted date, description, id, key (where key increments by 1) and company name

for i in range(len(adobe_jobs_title)):
    jobs.append({
        'title': adobe_jobs_title[i],
        'location': adobe_jobs_location[i],
        'link': adobe_jobs_link[i],
        'posted': adobe_jobs_posted[i],
        'description': adobe_jobs_description[i],
        'id': adobe_jobs_id[i],
        'company': 'Adobe'
    })

Get Able jobs

In [ ]:
able_url = 'https://boards.greenhouse.io/able'
able_response = requests.get(able_url, headers=headers)
able_soup = BeautifulSoup(able_response.content, 'html5lib')
# print(able_soup.prettify())

able_jobs = able_soup.find_all('a', attrs={"data-mapped":"true"})
# print(able_jobs)

# get the job title from able_jobs list stored in text
able_jobs_title = [job.text for job in able_jobs]
# print(able_jobs_title)

# get job link from attribute href
able_jobs_link = [greenhouse+job['href'] for job in able_jobs]
# print(able_jobs_link)

# get job location from span.location innertext
able_jobs_location = [able_soup.find_all('span', class_="location")[i].text for i in range(len(able_jobs))]
# print(able_jobs_location)

# writing results into a json file named as jobs.json in objects list with each object containing job title, location, link, posted date, description, id and key(we will use index of jobs[]).

for i in range(len(able_jobs_title)):
    jobs.append({
        'title': able_jobs_title[i],
        'location': able_jobs_location[i],
        'link': able_jobs_link[i],
        'posted': '',
        'description': '',
        'id': '',
        'company': 'Able'
    })


Get Affinidi jobs

In [31]:
affinidi_url = 'https://www.affinidi.com/careers'

# use selenium to get the page source
options = Options()
options.add_argument('--headless')
driver = Chrome(options=options)
driver.get(affinidi_url)

affinidi_soup = BeautifulSoup(driver.page_source, 'html5lib')

# get span.location innertext
affinidi_jobs_location = [affinidi_soup.find_all('span', class_="location")[i].text for i in range(len(affinidi_soup.find_all('span', class_="location")))]
print(affinidi_jobs_location)

NameError: name 'Options' is not defined